In [0]:
%tensorflow_version 2.x

In [0]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

--2020-04-22 13:52:24--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2020-04-22 13:52:24--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3420e967ecd0c2cef05f38326b.dl.dropboxusercontent.com/cd/0/inline/A2XGEef7JdlZk3vDsH9XYU8O0Qo3A_sgPofaJ9ypkO_CDzV_Ru3RJIiGvhqoZN8uFGmfMi187otfoftejJi5hG0s1vkCRO6NRMePcGO_VhR__A/file# [following]
--2020-04-22 13:52:25--  https://uc3420e967ecd0c2cef05f38326b.dl.dropboxusercontent.com/cd/0/inline/A2XGEef7JdlZk3vDsH9XYU8O0Qo3A_sgPofaJ9ypkO_CDzV_Ru3RJIiGvhqoZN8uFGmfMi187otfoftejJi5hG0s1vkCRO6NRMePcGO_VhR__A/file
Resol

In [0]:
!unzip /content/stacksample.zip?dl=0

Archive:  /content/stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

# **# Top10 Tags**

In [0]:
file='Tags.csv'

In [0]:
tags=pd.read_csv(file)

In [0]:
tags.head()
tags.shape

(3750994, 2)

In [0]:
top10tags=tags['Tag'].value_counts()[:10]

In [0]:
list10=top10tags.index.to_list()

In [0]:
tags_final=tags[tags['Tag'].isin(list10)]

In [0]:
tags_final.shape
#tags_final['Id'].value_counts().head(100)

(826739, 2)

In [0]:
df = tags_final.groupby(['Id'])['Tag'].apply(','.join).reset_index()

In [0]:
df[df['Id']==36283520]

,Id,Tag
625539,36283520,"javascript,android,jquery,ios"


In [0]:
df.shape

(706336, 2)

In [0]:
df['Tag']=df['Tag'].str.split(",")

# ***questions***

In [0]:
ques=pd.read_csv('Questions.csv',encoding='iso-8859-1')

In [0]:
ques.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [0]:
ques.drop(['OwnerUserId','CreationDate','ClosedDate','Score'],axis=1,inplace=True)

In [0]:
import re 

def rem_html_tags(body):
    regex = re.compile('<.*?>')
    return re.sub(regex, '',body)



In [0]:
ques['Body'] = ques['Body'].apply(rem_html_tags)
ques.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...
1,90,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...
2,120,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...
3,180,Function for creating color wheels,This is something I've pseudo-solved many time...
4,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...


In [0]:
total=pd.merge(ques, df,on='Id')
print(total.shape)
total.head()

(706336, 4)


,Id,Title,Body,Tag
0,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,[c#]
1,330,Should I use nested classes in this case?,I am working on a collection of classes used f...,[c++]
2,650,Automatically update version number,I would like the version property of my applic...,[c#]
3,930,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,[c#]
4,1010,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,[c#]


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model



Using TensorFlow backend.


In [0]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(total.Tag)
labels = multilabel_binarizer.classes_

In [0]:
train,test=train_test_split(total,test_size=0.25,random_state=24)

In [0]:
train.shape,test.shape

((529752, 4), (176584, 4))

In [0]:
X_train_t=train['Title']
X_train_b=train['Body']
y_train=multilabel_binarizer.transform(train['Tag'])
X_test_t=test['Title']
X_test_b=test['Body']
y_test=multilabel_binarizer.transform(test['Tag'])

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **Title Tokenize**

In [0]:
sent_lens_title=[]
for sent in train['Title']:
    sent_lens_title.append(len(word_tokenize(sent)))
max(sent_lens_title)

59

In [0]:
np.quantile(sent_lens_title,0.97)

18.0

In [0]:
max_len_title = 18
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
seq_train_t = tok.texts_to_sequences(X_train_t)

In [0]:
vocab_len_t=len(tok.index_word.keys())
vocab_len_t

82593

In [0]:
seq_matrix_train_t = sequence.pad_sequences(seq_train_t,maxlen=max_len_title)
seq_matrix_train_t

array([[    0,     0,     0, ...,    39, 13295, 35156],
       [    0,     0,     0, ...,   435,   326,    23],
       [    0,     0,     0, ...,   620,    14,  3998],
       ...,
       [    0,     0,     0, ...,    10,    74,  2835],
       [    0,     0,     0, ...,     2,    18,    81],
       [    0,     0,     0, ..., 14559,   794,   245]], dtype=int32)

In [0]:
seq_test_t = tok.texts_to_sequences(X_test_t)
seq_matrix_test_t = sequence.pad_sequences(seq_test_t,maxlen=max_len_title)

In [0]:
seq_matrix_train_t.shape,seq_matrix_test_t.shape,y_train.shape,y_test.shape

((529752, 18), (176584, 18), (529752, 10), (176584, 10))

# **Body Tokenize**

In [0]:
sent_lens_b=[]
for sent in train['Body']:
    sent_lens_b.append(len(word_tokenize(sent)))
max(sent_lens_b)

In [0]:
np.quantile(sent_lens_b,0.93)

In [0]:
max_len_b = 600
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_b)
sequences_train_b = tok.texts_to_sequences(X_train_b)

In [0]:
vocab_len_b =len(tok.index_word.keys())
vocab_len_b

1655486

In [0]:
sequences_matrix_train_b = sequence.pad_sequences(sequences_train_b,maxlen=max_len_b)
sequences_matrix_train_b

array([[     0,      0,      0, ...,    394,   2876, 881084],
       [     0,      0,      0, ...,      2,     56, 126139],
       [     0,      0,      0, ...,      9,   3514,   3577],
       ...,
       [     0,      0,      0, ...,     24,     72,    738],
       [     0,      0,      0, ...,    202,     62,     10],
       [     0,      0,      0, ...,    182,    203,     10]], dtype=int32)

In [0]:
sequences_test_b = tok.texts_to_sequences(X_test_b)
sequences_matrix_test_b = sequence.pad_sequences(sequences_test_b,maxlen=max_len_b)

In [0]:
seq_matrix_train_t.shape,sequences_matrix_train_b.shape,y_train.shape

((529752, 18), (529752, 600), (529752, 10))

In [0]:
seq_matrix_test_t.shape,sequences_matrix_test_b.shape,y_test.shape

((176584, 18), (176584, 600), (176584, 10))

In [0]:
def RNN():
    
    title_input = Input(name='title_input',shape=[max_len_title])
    title_Embed = Embedding(vocab_len_t+1,1000,input_length=max_len_title,mask_zero=True,name='title_Embed')(title_input)
    gru_out_t = GRU(200)(title_Embed)
    
    auxiliary_output = Dense(10, activation='sigmoid', name='aux_output')(gru_out_t)   
    
    
    body_input = Input(name='body_input',shape=[max_len_b]) 
    body_Embed = Embedding(vocab_len_b+1,250,input_length=max_len_b,mask_zero=True,name='body_Embed')(body_input)
    gru_out_b = GRU(100)(body_Embed)
    
    
    com = concatenate([gru_out_t, gru_out_b])
    
    
    dense1 = Dense(100,activation='relu')(com)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1) 
    dense2 = Dense(50,activation='relu')(bn)
    
    main_output = Dense(10, activation='sigmoid', name='main_output')(dense2)
    
    model = Model(inputs=[title_input, body_input],outputs=[main_output, auxiliary_output])
    return model

# New Section

In [0]:
model = RNN()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_input (InputLayer)        (None, 18)           0                                            
__________________________________________________________________________________________________
body_input (InputLayer)         (None, 600)          0                                            
__________________________________________________________________________________________________
title_Embed (Embedding)         (None, 18, 1000)     82594000    title_input[0][0]                
__________________________________________________________________________________________________
body_Embed (Embedding)          (None, 600, 250)     413871750   body_input[0][0]                 
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='adam',loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              metrics=['accuracy'])

In [0]:
results=model.fit({'title_input': seq_matrix_train_t, 'body_input': sequences_matrix_train_b},
          {'main_output': y_train, 'aux_output': y_train},
          validation_data=[{'title_input': seq_matrix_test_t, 'body_input': sequences_matrix_test_b},
          {'main_output': y_test, 'aux_output': y_test}],
          epochs=2, batch_size=500)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 529752 samples, validate on 176584 samples
Epoch 1/2
529752/529752 [==============================] - 6311s 12ms/step - loss: 0.3234 - main_output_loss: 0.1486 - aux_output_loss: 0.1748 - main_output_accuracy: 0.9442 - aux_output_accuracy: 0.9370 - val_loss: 0.2244 - val_main_output_loss: 0.0889 - val_aux_output_loss: 0.1355 - val_main_output_accuracy: 0.9671 - val_aux_output_accuracy: 0.9488
Epoch 2/2
529752/529752 [==============================] - 5994s 11ms/step - loss: 0.2063 - main_output_loss: 0.0842 - aux_output_loss: 0.1221 - main_output_accuracy: 0.9694 - aux_output_accuracy: 0.9533 - val_loss: 0.2115 - val_main_output_loss: 0.0784 - val_aux_output_loss: 0.1331 - val_main_output_accuracy: 0.9706 - val_aux_output_accuracy: 0.9495


In [0]:
results

In [0]:
(predicted_main, predicted_aux)=model.predict({'title_input': seq_matrix_test_t, 'body_input': sequences_matrix_test_b},verbose=1)


176584/176584 [==============================] - 898s 5ms/step


In [0]:
predicted_main[1:10]

In [0]:
from sklearn.metrics import classification_report,f1_score,roc_auc_score

In [0]:
print(f1_score(y_test,predicted_main>.55,average='samples'))

0.8765240824277882


In [0]:
print(roc_auc_score(y_test,predicted_main))

0.985100743453701


In [0]:
print(classification_report(y_test,predicted_main>0.55))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95     22702
           1       0.93      0.88      0.90     25377
           2       0.91      0.86      0.88     11810
           3       0.85      0.41      0.56     14809
           4       0.96      0.92      0.94     11619
           5       0.94      0.82      0.87     28612
           6       0.86      0.73      0.79     31091
           7       0.84      0.79      0.81     19641
           8       0.95      0.90      0.92     24507
           9       0.97      0.95      0.96     16161

   micro avg       0.92      0.82      0.87    206329
   macro avg       0.92      0.82      0.86    206329
weighted avg       0.92      0.82      0.86    206329
 samples avg       0.91      0.87      0.88    206329



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
predicted_main[1024].round(decimals = 2)

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
test.iloc[1024]

Id                                                22201820
Title    Redirect page in web.config for a certain exce...
Body     I have a MVC solution, which works with author...
Tag                                                   [c#]
Name: 353422, dtype: object

In [0]:
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [0]:
model.save('./tags.h5')